## Notebook to keep track of all the subjects

- Global manifest: `participants.csv`. This should be placed within `<mr_proc_dataset>/tabular/demographics`. This should consists following columns:
    - `participant_id`, `age`, `sex`, `group` generated by study coordinator Google sheets / database
    - `BIDS_id` column used by Heudiconv and other mr_proc pipelines that maps `participant_id` 

In [1]:
import pandas as pd
import numpy as np

### Paths

In [14]:
tabular_dir = "/home/nikhil/projects/Parkinsons/QPN_processing/tabular/"

previous_release = "release_Sept2022"
current_release = "release_Nov2022"

# Previous mr_proc manifest
previous_mr_proc_manifest_csv = f"{tabular_dir}{previous_release}/mr_proc_manifest.csv"

# Current mr_proc manifest
current_mr_proc_manifest_csv = f"{tabular_dir}{current_release}/mr_proc_manifest.csv"

# Current recruit manifest
current_recruit_manifest_xls = f"{tabular_dir}{current_release}/QPN_Imaging_Codes.xlsx"

# Current BIDS manifest
current_bids_manifest_tsv = f"{tabular_dir}{current_release}/bids_participants.tsv"

# Current COPN LORIS manifest
COPN_LORIS_csv = f"{tabular_dir}{current_release}/COPN_LORIS_manifest_Nov2022.csv"

# Current Neuropsych CSV
neuropsych_xls = f"{tabular_dir}/{current_release}/BD_RPQ_UPDATE_Neuropsy.xlsx"


### Read recruitment manifest from previous release

In [47]:
previous_recruit_manifest_df = pd.read_csv(previous_mr_proc_manifest_csv)

mr_proc_participants_previous = previous_recruit_manifest_df["participant_id"].dropna().unique()
n_mr_proc_participants_previous = len(mr_proc_participants_previous)

print(f"number of participants from previous mr_proc release: {n_mr_proc_participants_previous}")
previous_recruit_manifest_df.head()

number of participants from previous mr_proc release: 296


,participant_id,age,sex,group,bids_id
0,MNI0056,79.0,M,Parkinson,sub-MNI0056D864854
1,MNI0058,57.0,M,Parkinson,sub-MNI0058D197308
2,MNI0059,63.0,M,Parkinson,NaN
3,MNI0068,69.0,M,Parkinson,sub-MNI0068D842090
4,MNI0079,50.0,F,Parkinson,sub-MNI0079D760662


### Read latest recruitment manifest

In [73]:
sheet_name = "recruit_manifest"
column_list = ["participant_id","sex","subj_dob","group","visit_01","visit_02"]

current_recruit_manifest_df = pd.read_excel(current_recruit_manifest_xls,sheet_name=sheet_name, engine='openpyxl',
usecols=column_list)

current_recruit_manifest_df = current_recruit_manifest_df.dropna(how="all", axis=0)

mr_proc_participants_current = current_recruit_manifest_df["participant_id"].dropna().unique()
n_mr_proc_participants_current = len(mr_proc_participants_current)
print(f"number of participants from current mr_proc release: {n_mr_proc_participants_current}")

current_recruit_manifest_df["dob"] = pd.to_datetime(current_recruit_manifest_df['subj_dob'],errors='coerce')
current_recruit_manifest_df["visit_01"] = pd.to_datetime(current_recruit_manifest_df['visit_01'],errors='coerce')
current_recruit_manifest_df["visit_02"] = pd.to_datetime(current_recruit_manifest_df['visit_02'],errors='coerce')

current_recruit_manifest_df.head()

number of participants from current mr_proc release: 208


,participant_id,sex,subj_dob,group,visit_01,visit_02,dob
0,MNI0056,M,21-05-1942,PD,2021-08-18,NaT,1942-05-21
1,MNI0058,M,14-03-1964,PD,2021-08-18,NaT,1964-03-14
2,MNI0068,M,1952-08-05 00:00:00,PD,2021-08-27,NaT,1952-08-05
3,MNI0079,F,25-11-1971,PD,2021-12-22,NaT,1971-11-25
4,MNI0103,M,17-08-1939,PD,2021-08-11,NaT,1939-08-17


### Get new participants

In [74]:
partcipants_additions = list(set(mr_proc_participants_current) - set(mr_proc_participants_previous))

print(f"number of new participants: {len(partcipants_additions)}")

number of new participants: 27


### Read LORIS manifest 
- To generate BIDS ID

In [75]:
loris_df = pd.read_csv(COPN_LORIS_csv)
participants_loris = set(loris_df["PSCID"].str.strip())
n_participants_loris = len(participants_loris)
print(f"Number of total LORIS (imaging) participants: {n_participants_loris}")

Number of total LORIS (imaging) participants: 2290


### Read latest BIDS tsv

In [76]:
PSCID_start = 4
PSCID_end = 11

current_bids_df = pd.read_csv(current_bids_manifest_tsv,sep="\t")
current_bids_df["bids_id"] = current_bids_df["participant_id"].copy()
current_bids_df["participant_id"] = current_bids_df["bids_id"].str[PSCID_start:PSCID_end]

n_bids_current = len(current_bids_df["participant_id"].dropna().unique())
print(f"number of participants from current mr_proc release: {n_bids_current}")

current_bids_df.head()


number of participants from current mr_proc release: 197


,participant_id,age,sex,group,bids_id
0,PD01369,55,M,control,sub-PD01369D713546
1,MNI0103,82,M,control,sub-MNI0103D369057
2,PD00215,48,M,control,sub-PD00215D795669
3,MNI0056,79,M,control,sub-MNI0056D864854
4,PD00296,51,M,control,sub-PD00296D884528


### Generate current_mr_proc_manifest_csv

In [79]:
current_mr_proc_manifest_df = pd.merge(current_recruit_manifest_df[["participant_id","sex","dob","group","visit_01","visit_02"]],
                                        current_bids_df[["participant_id","bids_id"]],
                                        on="participant_id", how="left")

print(f"current_mr_proc_manifest_len: {len(current_mr_proc_manifest_df)}")

current_mr_proc_manifest_df.head()

current_mr_proc_manifest_len: 208


,participant_id,sex,dob,group,visit_01,visit_02,bids_id
0,MNI0056,M,1942-05-21,PD,2021-08-18,NaT,sub-MNI0056D864854
1,MNI0058,M,1964-03-14,PD,2021-08-18,NaT,sub-MNI0058D197308
2,MNI0068,M,1952-08-05,PD,2021-08-27,NaT,sub-MNI0068D842090
3,MNI0079,F,1971-11-25,PD,2021-12-22,NaT,sub-MNI0079D760662
4,MNI0103,M,1939-08-17,PD,2021-08-11,NaT,sub-MNI0103D369057


### Save update CSV

In [81]:
save_current_mr_proc_manifest = False
if save_current_mr_proc_manifest:
    print(f"Saving new mr_proc manifest here: {current_mr_proc_manifest_csv}")
    current_mr_proc_manifest_df.to_csv(current_mr_proc_manifest_csv,index=None)

Saving new mr_proc manifest here: /home/nikhil/projects/Parkinsons/QPN_processing/tabular/release_Nov2022/mr_proc_manifest.csv


### BIDS participants missing from global participant list

### TODO: Check participants in neuropsych
- `QPN_Imaging_Codes` should have neuropsych for mr cohort. 


In [69]:
sheet_names = ["Parkinson patients", "Control"]
subject_id_list = []
dx_list = []

neuropsy_df_concat = pd.DataFrame()

for sheet_name in sheet_names:
    print(sheet_name)

    neuropsy_df = pd.read_excel(demographics_file,sheet_name=sheet_name, engine='openpyxl',header=1)
    neuropsy_df = neuropsy_df.dropna(how='all')

    # Get rid of "/T1" from subject ID (timepoint)
    neuropsy_df["Patient #"] = neuropsy_df["Patient #"].str.split("/",expand=True)[0]
    subject_ids = list(neuropsy_df["Patient #"].values)
    print(f"Number of participants: {len(subject_ids)}")

    subject_id_list +=subject_ids
    dx_list += list(np.tile(sheet_name,len(subject_ids)))

    neuropsy_df_concat = neuropsy_df_concat.append(neuropsy_df)

print(f"Total number of neuropsy subjects: {len(subject_id_list)}")
neuropsy_df_concat.head()

Parkinson patients
Number of participants: 292
Control
Number of participants: 53
Total number of neuropsy subjects: 345


,Patient #,Administered by,Parkinson Disease or Control subject,Sex (1=men; 2=women),Language,Date of assessment,Date of birth,Age at time of assessment,Date of apparition of first symptom,Date of diagnosis,...,"Stroop - D-Kefs, COLORS (condition 1): Time (sec) (Raw score)",BNT sans indices,"Subjective Complaint (yes/no) DO you have feel that you issues with your memory, judgement, concentration, planning, etc?",Comments,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80
0,PD00209,Sabrina,Parkinson,1.0,french,no record,21/05/1960,59.0,,2013,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PD00119,Erika,Parkinson,1.0,english,2017-07-12 00:00:00,12/11/1951,66.0,,2008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PD00820,Erika,Parkinson,1.0,french,2017-08-01 00:00:00,20/08/1947,69.0,,2009,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PD00262,Erika,Parkinson,2.0,999,2017-08-12 00:00:00,26/07/1947,71.0,2011,2011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PD00523,Erika,Parkinson,2.0,999,2017-12-14 00:00:00,01/10/1933,84.0,,999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
save_csv = False

partcipants_df = neuropsy_df_concat[["Patient #","Parkinson Disease or Control subject","Sex (1=men; 2=women)","Age at time of assessment"]].copy()

partcipants_df = partcipants_df.rename(columns={"Patient #":"participant_id",
                                                "Parkinson Disease or Control subject":"group",
                                                "Sex (1=men; 2=women)":"sex",
                                                "Age at time of assessment":"age"})

partcipants_df = partcipants_df.dropna(how="all")

partcipants_df['participant_id'] = partcipants_df['participant_id'].str.strip()
partcipants_df["age"] = np.round(partcipants_df["age"],1)
partcipants_df["sex"] = partcipants_df["sex"].replace({1:"M",2:"F"})

partcipants_df = partcipants_df[["participant_id","age","sex","group"]]
partcipants_df = partcipants_df.sort_values(by=["participant_id","age"])

# Keep only unique participant ids (age refers to the baseline visit)
print(f"number of participants entries (includes duplicates from multiple visits): {len(partcipants_df)}")
partcipants_df = partcipants_df.drop_duplicates(subset=["participant_id"],keep="first")
print(f"number of unique participants: {len(partcipants_df)}")

if save_csv:
    partcipants_df.to_csv("../metadata/participants.csv", index=False)

partcipants_df.head()


number of participants entries (includes duplicates from multiple visits): 338
number of unique participants: 306


,participant_id,age,sex,group
174,MNI0056,79.0,M,Parkinson
183,MNI0058,57.0,M,Parkinson
175,MNI0059,63.0,M,Parkinson
182,MNI0068,69.0,M,Parkinson
227,MNI0079,50.0,F,Parkinson


In [71]:
partcipants_df = pd.merge(partcipants_df,bids_df, on="participant_id", how="left")
n_total_participants = len(partcipants_df["participant_id"].dropna().unique())
n_bids_participants = len(partcipants_df["bids_id"].dropna().unique())

print(f"After updated merge: number of total participants: {n_total_participants} BIDS participants: {n_bids_participants}")

partcipants_df.head()

After updated merge: number of total participants: 306 BIDS participants: 144


,participant_id,age,sex,group,bids_id
0,MNI0056,79.0,M,Parkinson,sub-MNI0056D864854
1,MNI0058,57.0,M,Parkinson,sub-MNI0058D197308
2,MNI0059,63.0,M,Parkinson,NaN
3,MNI0068,69.0,M,Parkinson,sub-MNI0068D842090
4,MNI0079,50.0,F,Parkinson,sub-MNI0079D760662
